In [1]:
import pandas as pd
import numpy
import pickle
from sklearn.preprocessing import MinMaxScaler

# Carga el dataset procesado
df = pd.read_csv('../../data/processed/features_for_model.csv')

with open('../../artifacts/feature_eng_configs.pkl', 'rb') as f:
    feature_eng_configs = pickle.load(f)

with open('../../artifacts/mm_scaler.pkl', 'rb') as f:
    mm_scaler = pickle.load(f)

# Imputar valores en las columnas correspondientes
cols_imputacion = ["Glucose", "BloodPressure", "BMI"]
for col in cols_imputacion:
    df[col] = df[col].replace(0, feature_eng_configs[f"{col}_imputed_value"]).astype(int)

# Aplicar la estandarización a las características
df_scaled = pd.DataFrame(mm_scaler.transform(df), columns=df.columns)

# Verificación
print(df_scaled.head())

   Pregnancies   Glucose  BloodPressure       BMI  DiabetesPedigreeFunction  \
0     0.035503 -0.459259      -0.727273 -0.514286                  0.118524   
1     0.011834 -0.459259      -0.727273 -0.514286                 -0.051464   
2     0.011834 -0.459259      -0.727273 -0.514286                 -0.045436   
3     0.047337 -0.459259      -0.727273 -0.514286                  0.375313   
4     0.041420 -0.459259      -0.727273 -0.514286                 -0.014091   

        Age  Outcome  
0 -0.446125      0.0  
1 -0.456522      0.0  
2 -0.456522      0.0  
3 -0.450378      0.0  
4 -0.442817      0.0  


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
# Dividir el dataset en X (características) y y (target)
X = df_scaled.drop('Outcome', axis=1)
y = df_scaled['Outcome']

# Dividir los datos en conjunto de entrenamiento y prueba (con los datos ya procesados)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificación
print(X_train.shape, X_test.shape)

(123, 6) (31, 6)


In [3]:
# Entren el modelo

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Crear el modelo
model_lr = LogisticRegression(random_state=42)

# Entrenar el modelo
model_lr.fit(X_train, y_train)

# Realizar predicciones
y_pred_lr = model_lr.predict(X_test)

# Evaluar el modelo
print("Logistic Regression - Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression - Classification Report:\n", classification_report(y_test, y_pred_lr))
print("Logistic Regression - Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lr))


Logistic Regression - Accuracy: 0.5161290322580645
Logistic Regression - Classification Report:
               precision    recall  f1-score   support

         0.0       0.52      1.00      0.68        16
         1.0       0.00      0.00      0.00        15

    accuracy                           0.52        31
   macro avg       0.26      0.50      0.34        31
weighted avg       0.27      0.52      0.35        31

Logistic Regression - Confusion Matrix:
 [[16  0]
 [15  0]]


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
# Probar diferentes configuraciones para Logistic Regression
model_lr_1 = LogisticRegression(random_state=42, max_iter=200)
model_lr_1.fit(X_train, y_train)
y_pred_lr_1 = model_lr_1.predict(X_test)
print("Logistic Regression Config 1 - Accuracy:", accuracy_score(y_test, y_pred_lr_1))

model_lr_2 = LogisticRegression(random_state=42, solver='liblinear')
model_lr_2.fit(X_train, y_train)
y_pred_lr_2 = model_lr_2.predict(X_test)
print("Logistic Regression Config 2 - Accuracy:", accuracy_score(y_test, y_pred_lr_2))


Logistic Regression Config 1 - Accuracy: 0.5161290322580645
Logistic Regression Config 2 - Accuracy: 0.5161290322580645


In [5]:
# Guardar el modelo entrenado
with open('../../artifacts/logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(model_lr, f)

